
# ビームサーチ

:label: `sec_beam-search`

 :numref: `sec_seq2seq`では、エンコーダ/デコーダ アーキテクチャと、それらをエンドツーエンドでトレーニングするための標準的な手法を紹介しました。ただし、テスト時の予測に関しては、*貪欲な*戦略のみについて言及しました。この戦略では、あるタイム ステップで予測したことが判明するまで、次に来る予測確率が最も高いトークンを各タイム ステップで選択します。特別なシーケンスの終わり「&amp;lt;eos&amp;gt;」トークン。このセクションでは、この*貪欲な検索*戦略を形式化し、実践者が遭遇しがちないくつかの問題を特定することから始めます。次に、この戦略を 2 つの代替案、つまり*網羅的探索*(例示的だが実用的ではない) と*ビーム探索*(実際の標準的な方法) と比較します。

 :numref: `sec_seq2seq`から規則を借用して、数学的表記法を設定することから始めましょう。任意のタイム ステップ $t&#39;$ で、デコーダは、語彙内の各トークンがシーケンスで次に来る確率 (前のトークン $y_1 を条件とした $y_{t&#39;+1}$ の可能性の高い値) を表す予測を出力します。 \ldots, y_{t&#39;}$ およびコンテキスト変数 $\mathbf{c}$ は、入力シーケンスを表すためにエンコーダーによって生成されます。計算コストを定量化するには、出力語彙 (含む) を $\mathcal{Y}$ で示します。特殊なシーケンス終了トークン "&amp;lt;eos&gt;")。また、出力シーケンスのトークンの最大数を $T&#39;$ として指定しましょう。私たちの目標は、すべての $\mathcal{O から理想的な出力を検索することです。 }(\left|\mathcal{Y}\right|^{T&#39;})$ 可能な出力シーケンス。「&amp;lt;eos&amp;gt;」トークンが発生した後に後続のトークンがないため、これは個別の出力の数をわずかに過大評価していることに注意してください。ただし、ここでの目的では、この数値は検索スペースのサイズを大まかに把握します。

## 貪欲な検索

:numref: `sec_seq2seq`からの単純な*貪欲検索*戦略を考えてみましょう。ここで、任意のタイム ステップ $t&#39;$ で、$\mathcal{Y}$ から条件付き確率が最も高いトークンを選択するだけです。つまり、

 $$y_{t&#39;} = \operatorname*{argmax} *{y \in \mathcal{Y}} P(y \mid y_1, \ldots, y* {t&#39;-1}, \mathbf{c}).$ $

モデルが「&amp;lt;eos&amp;gt;」を出力すると、 (または最大長 $T&#39;$ に達すると) 出力シーケンスが完了します。

この戦略は合理的に見えるかもしれませんが、実際にはそれほど悪くありません。計算負荷がどれほど低いかを考えると、これ以上の利益を得ることは難しいでしょう。ただし、効率を少し脇に置くと、(貪欲に選択された)*最も可能性の高いトークンのシーケンス**ではなく、最も可能性の高いシーケンス*を検索する方が合理的であるように思えるかもしれません。これら 2 つのオブジェクトはまったく異なる可能性があることがわかります。最も可能性の高いシーケンスは、式 $\prod_{t&#39;=1}^{T&#39;} P(y_{t&#39;} \mid y_1, \ldots, y_{t&#39;-1}, \mathbf{ c})$。機械翻訳の例では、デコーダーが基礎となる生成プロセスの確率を本当に復元した場合、最も可能性の高い翻訳が得られます。残念ながら、欲張り検索でこのシーケンスが得られるという保証はありません。

例を挙げて説明しましょう。 4 つのトークン「A」、「B」、「C」、「&amp;lt;eos&amp;gt;」があるとします。出力辞書に含まれます。 :numref: `fig_s2s-prob1`では、各タイム ステップの下の 4 つの数字は、「A」、「B」、「C」、および「&lt;eos&gt;」を生成する条件付き確率を表します。それぞれそのタイムステップで。 

![](../img/s2s-prob1.svg) :ラベル: `fig_s2s-prob1`

各タイム ステップで、貪欲検索は条件付き確率が最も高いトークンを選択します。したがって、出力シーケンス「A」、「B」、「C」、および「&lt;eos&gt;」は、が予測されます (:numref: `fig_s2s-prob1` )。この出力シーケンスの条件付き確率は $0.5\times0.4\times0.4\times0.6 = 0.048$ です。

次に、 :numref: `fig_s2s-prob2`の別の例を見てみましょう。 :numref: `fig_s2s-prob1`とは異なり、タイム ステップ 2 では、 *2 番目*に高い条件付き確率を持つ :numref: `fig_s2s-prob2`のトークン "C" を選択します。 

![](../img/s2s-prob2.svg) :ラベル: `fig_s2s-prob2`

タイム ステップ 3 のベースとなるタイム ステップ 1 および 2 の出力サブシーケンスは、:numref: `fig_s2s-prob1`の「A」および「B」から、:numref: `fig_s2s-prob2`の「A」および「C」に変更されているため、 `fig_s2s-prob2` 、タイム ステップ 3 での各トークンの条件付き確率も :numref: `fig_s2s-prob2`で変更されています。タイム ステップ 3 でトークン "B" を選択するとします。タイム ステップ 4 は、最初の 3 つのタイム ステップ "A"、"C"、および "B" での出力サブシーケンスが条件となっており、これは "A" とは異なります。 :numref: `fig_s2s-prob1`の「B」、および「C」。したがって、 :numref: `fig_s2s-prob2`のタイム ステップ 4 で各トークンを生成する条件付き確率も、 :numref: `fig_s2s-prob1`の条件付き確率とは異なります。その結果、出力シーケンス「A」、「C」、「B」、および「&lt;eos&gt;」の条件付き確率は次のようになります。 :numref: `fig_s2s-prob2`は $0.5\times0.3 \times0.6\times0.6=0.054$ で、これは :numref: `fig_s2s-prob1`の貪欲検索の値よりも大きくなります。この例では、出力シーケンス「A」、「B」、「C」、および「&lt;eos&gt;」貪欲探索によって得られた配列は最適な配列ではありません。

## 徹底的な検索

目標が最も可能性の高いシーケンスを取得することである場合は、*網羅的検索*の使用を検討できます。つまり、考えられるすべての出力シーケンスを条件付き確率で徹底的に列挙し、予測確率が最も高いものを出力します。

これにより確かに私たちが望むものは得られますが、$\mathcal{O}(\left|\mathcal{Y}\right|^{T&#39;})$ という法外な計算コストがかかり、配列の長さが指数関数的に増加します。そして語彙のサイズによって与えられる膨大な基礎を備えています。たとえば、$|\mathcal{Y}|=10000$ および $T&#39;=10$ の場合、$10000^{10} = 10^{40}$ シーケンスを評価する必要があります。これらは実際のアプリケーションに比べれば小さい数字ですが、すでに予測可能なコンピューターの能力を超えています。一方、貪欲検索の計算コストは​​ $\mathcal{O}(\left|\mathcal{Y}\right|T&#39;)$ です。これは奇跡的に安いですが、最適とは程遠いです。たとえば、$|\mathcal{Y}|=10000$ および $T&#39;=10$ の場合、$10000\times10=10^5$ シーケンスを評価するだけで済みます。

## ビームサーチ

配列デコード戦略はスペクトル上に存在し、貪欲検索の効率と網羅的検索の最適性の間で*ビーム*検索が妥協点をとっているとみなすことができます。ビーム探索の最も単純なバージョンは、単一のハイパーパラメータである*ビーム サイズ*$k$ によって特徴付けられます。タイムステップ 1 では、予測確率が最も高い $k$ トークンを選択します。それらのそれぞれは、それぞれ $k$ 候補出力シーケンスの最初のトークンになります。後続の各タイム ステップでは、前のタイム ステップでの $k$ 候補出力シーケンスに基づいて、$k\left|\mathcal{Y}\right| から最も高い予測確率を持つ $k$ 候補出力シーケンスを選択し続けます。 $ 可能な選択肢。 

![](../img/beam-search.svg) :label: `fig_beam-search`

 :numref: `fig_beam-search`例を使用してビーム検索のプロセスを示します。出力語彙には $\mathcal{Y} = {A, B, C, D, E}$ の 5 つの要素のみが含まれており、そのうちの 1 つが「&amp;lt;eos&amp;gt;」であるとします。ビーム サイズを 2、出力シーケンスの最大長を 3 とします。タイム ステップ 1 で、最も高い条件付き確率 $P(y_1 \mid \mathbf{c})$ を持つトークンが $A$ と $ であると仮定します。 Cドル。タイムステップ 2 では、すべての $y_2 \in \mathcal{Y},$ について計算します。

 $$\begin{aligned}P(A, y_2 \mid \mathbf{c}) = P(A \mid \mathbf{c})P(y_2 \mid A, \mathbf{c}),\ P(C , y_2 \mid \mathbf{c}) = P(C \mid \mathbf{c})P(y_2 \mid C, \mathbf{c}),\end{aligned}$$

そして、これら 10 個の値のうち最大の 2 つを選択します。$P(A, B \mid \mathbf{c})$ と $P(C, E \mid \mathbf{c})$ とします。次に、タイム ステップ 3 で、すべての $y_3 \in \mathcal{Y}$ に対して、次の計算を行います。

 $$\begin{aligned}P(A, B, y_3 \mid \mathbf{c}) = P(A, B \mid \mathbf{c})P(y_3 \mid A, B, \mathbf{c} ),\P(C, E, y_3 \mid \mathbf{c}) = P(C, E \mid \mathbf{c})P(y_3 \mid C, E, \mathbf{c}),\end {整列}$$

そして、これら 10 個の値の中から最大の 2 つを選択します。$P(A, B, D \mid \mathbf{c})$ と $P(C, E, D \mid \mathbf{c})$ とします。の場合、6 つの候補出力シーケンスが得られます。 (i) $A$; (ii) $C$; (iii) $A$、$B$。 (iv) $C$、$E$。 (v) $A$、$B$、$D$。 (vi) $C$、$E$、$D$。

最終的に、これら 6 つのシーケンスに基づいて最終候補出力シーケンスのセットを取得します (たとえば、「&lt;eos&gt;」以降を含む部分を破棄します)。次に、次のスコアの最も高いシーケンスを出力シーケンスとして選択します。

 $$ \frac{1}{L^\alpha} \log P(y_1, \ldots, y_{L}\mid \mathbf{c}) = \frac{1}{L^\alpha} \sum_{t &#39;=1}^L \log P(y_{t&#39;} \mid y_1, \ldots, y_{t&#39;-1}, \mathbf{c}),$$ :eqlabel: `eq_beam-search-score`

ここで、$L$ は最終候補シーケンスの長さであり、$\alpha$ は通常 0.75 に設定されます。長いシーケンスには :eqref: `eq_beam-search-score`の合計に多くの対数項が含まれるため、分母の項 $L^\alpha$ は長いシーケンスにペナルティを与えます。

ビーム探索の計算コストは​​ $\mathcal{O}(k\left|\mathcal{Y}\right|T&#39;)$ です。この結果は、貪欲検索と網羅的検索の中間に位置します。貪欲探索は、ビーム サイズが 1 に設定されている場合に発生するビーム探索の特殊なケースとして扱うことができます。

## まとめ

シーケンス検索戦略には、貪欲検索、網羅的検索、およびビーム検索が含まれます。ビームサーチは、ビームサイズを柔軟に選択できるため、精度と計算コストの間のトレードオフを実現します。

## 演習
1. 網羅的探索を特殊なタイプのビーム探索として扱うことはできますか?なぜ、あるいはなぜそうではないのでしょうか？
1.  :numref: `sec_seq2seq`の機械翻訳問題にビーム検索を適用します。ビームサイズは翻訳結果と予測速度にどのような影響を与えますか?
1.  :numref: `sec_rnn-scratch`でユーザーが指定したプレフィックスに続くテキストを生成するために、言語モデリングを使用しました。どのような種類の検索戦略が使用されますか?改善してもらえますか？

[ディスカッション](https://discuss.d2l.ai/t/338)
